In [ ]:
import pickle

In [ ]:
import ast

In [ ]:
out_dir = '/content/drive/MyDrive/human_eval/assignment_2099204_export-2'

In [ ]:
import os

csv_paths = []
for fol in os.listdir(out_dir):
  if not os.path.isdir(os.path.join(out_dir, fol)):
    continue
  for c in os.listdir(os.path.join(out_dir, fol)):
    if '.csv' in c:
      csv_paths.append(os.path.join(out_dir, fol,c))

In [ ]:
import pandas as pd

In [ ]:
cem = pickle.load(open('/content/drive/MyDrive/human_eval/MRD_prediction/mrd_cem.p', 'rb'))
clip = pickle.load(open('/content/drive/MyDrive/human_eval/MRD_prediction/mrd_clip.p', 'rb'))
gpt3 = pickle.load(open('/content/drive/MyDrive/human_eval/MRD_prediction/mrd_gpt3.p', 'rb'))
lm = pickle.load(open('/content/drive/MyDrive/human_eval/MRD_prediction/mrd_lm.p', 'rb'))
objects = list(cem.keys())

In [ ]:
obj2control = pickle.load(open('/content/drive/MyDrive/human_eval/range_ipynb/obj2control.p', 'rb'))

In [ ]:
cem[objects[0]]

In [ ]:
cem_rates = {obj: {} for obj in objects}
cem_list = []
clip_rates =  {obj: {} for obj in objects}
clip_list =[]
gpt3_rates =  {obj: {} for obj in objects}
gpt3_list = []
lm_rates =  {obj: {} for obj in objects}
lm_list =[]

k = 3
failed = 0
total = 0
for f in csv_paths:
  df = pd.read_csv(f, delimiter='\t')
  for i,row in df.iterrows():
    try:
      obj = row['obj']
    except:
      print(f)
      break
    vals = ast.literal_eval(row['property_values'])
    if obj2control[obj] in vals and vals[obj2control[obj]] > 3:
      failed+=1
      continue
    total += 1
    for p,v in vals.items():
      p = p.strip()
      if v == 2.5:
        continue
      if p in cem[obj][:k]:
        cem_list.append(v)
        if p in cem_rates[obj]:
          cem_rates[obj][p].append(v)
        else:
           cem_rates[obj][p] = [v]
      if p in clip[obj][:k]:
        clip_list.append(v)
        if p in clip_rates[obj]:
          clip_rates[obj][p].append(v)
        else:
          clip_rates[obj][p] = [v]
      if p in gpt3[obj][:k]:
        gpt3_list.append(v)
        if p in gpt3_rates[obj]:
          gpt3_rates[obj][p].append(v)  
        else:
             gpt3_rates[obj][p]= [v]
      if p in lm[obj][:k]:
        lm_list.append(v)
        if p in lm_rates[obj]:
          lm_rates[obj][p].append(v)  
        else:
          lm_rates[obj][p] = [v]  
print(total)
print(failed) 


764
185


In [ ]:
import numpy as np
print(np.mean(lm_list))
print(np.mean(gpt3_list))
print(np.mean(clip_list))
print(np.mean(cem_list))

2.278221137077084
3.1535276073619634
2.694066305818674
2.7823006447234477


In [ ]:
def kendall_tau(rates):
  annotators  = [{}, {}, {}]
  for obj, properties in rates.items():
    for i in range(len(annotators)):
      annotators[i][obj] = {}
    for p, val in properties.items():
      # if len(val) < 3:
      #   print(obj, p)
      for i in range(min(3, len(val))):
         annotators[i][obj][p] = val[i]
  
  annotators_order = [{}, {}, {}]
  for i in range(len(annotators)):
    for obj, properties in annotators[i].items():
      for p in properties:
        for p_ in properties:
          if properties[p] <= properties[p_]:
              annotators_order[i][(p, p_)] = 1
          else:
              annotators_order[i][(p, p_)] = -1

    
  conc = {(0,1):0, (0,2):0, (1,2):0}
  disc = {(0,1):0, (0,2):0, (1,2):0}
  total = len(annotators_order[0])
  for itm in annotators_order[0].keys():
    for i,j in [(0,1), (0,2), (1,2)]:
      if itm in annotators_order[i] and itm in annotators_order[j]:
        if  annotators_order[i][itm] == annotators_order[j][itm]:
            conc[(i,j)]+=1
        else:
            disc[(i,j)]+=1
  s = 0
  for i,j in [(0,1), (0,2), (1,2)]:
    s+=(conc[(i,j)]-disc[(i,j)])/(conc[(i,j)]+disc[(i,j)])
  print(s/3)

In [ ]:
kendall_tau(gpt3_rates)

0.4059264589546128


In [ ]:
import numpy as np

def mean_rates(rates):
  obj2mean = {}
  for obj, properties in rates.items():
    mean = []
    if len(properties) == 0:
      continue
    for p, scores in properties.items():
      mean.append(np.mean(scores))
    obj2mean[obj] = np.mean(mean)
  return np.mean(list(obj2mean.values()))


In [ ]:
print(mean_rates(cem_rates))
print(mean_rates(gpt3_rates))

2.6833806729264476
3.0749116586538463


In [ ]:
students =['btpokin@seas.upenn.edu', 'jiatais@seas.upenn.edu', 'yueyang1@seas.upenn.edu', 'chause@seas.upenn.edu', 'xsamore@seas.upenn.edu', 'hdejong@seas.upenn.edu', 'avivm@seas.upenn.edu', 'admaga@seas.upenn.edu', 'wiwhi@seas.upenn.edu', 'jiatais@seas.upenn.edu', 'chenmcao@seas.upenn.edu', 'wingli@seas.upenn.edu', 'zdw466@seas.upenn.edu', 'xdong46@seas.upenn.edu', 'admaga@seas.upenn.edu', 'jashshah@seas.upenn.edu', 'cnguyenc@seas.upenn.edu', 'calmard@seas.upenn.edu', 'jihongl@seas.upenn.edu']

In [ ]:
len(set(students))

17